# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!


  1. Use RAGAS to Generate Synthetic Data
  2. Load them into a LangSmith Dataset
  3. Evaluate our RAG chain against the synthetic test data
  4. Make changes to our pipeline
  5. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [34]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/allantan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/allantan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [35]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [36]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"PSI - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [37]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [49]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [50]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [51]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [53]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 40, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [54]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/40 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/40 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/120 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 920)

We can save and load our knowledge graphs as follows.

In [65]:
kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
bills_data_kg

KnowledgeGraph(nodes: 40, relationships: 920)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [66]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [68]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

</div>


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

* Answer here
</span>


Finally, we can use our `TestSetGenerator` to generate our testset!

In [69]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of SENATE in AI regulation?,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The SENATE is involved in regulating the devel...,single_hop_specifc_query_synthesizer
1,Why is Georgetown University important for AI ...,[AI presents enormous opportunities for the Ph...,The context does not provide specific informat...,single_hop_specifc_query_synthesizer
2,What does AIRA mean?,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The Artificial Intelligence Regulation Act (AI...,single_hop_specifc_query_synthesizer
3,What is AGI and how does it relate to AI devel...,"[1 \na) Promote innovation, technological adva...",The context states that the Act shall regulate...,single_hop_specifc_query_synthesizer
4,What does AGI stand for in artificial intellig...,[1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11\n12\n13\n1...,Artificial General Intelligence (AGI) refers t...,single_hop_specifc_query_synthesizer
5,How does the Philippine AI regulation address ...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,The Philippine AI regulation emphasizes the pr...,multi_hop_abstract_query_synthesizer
6,how do risk-based classification and transpare...,[<1-hop>\n\n1 \nd) Take full responsibility fo...,The context indicates that providing a clear m...,multi_hop_abstract_query_synthesizer
7,"How do the principles of transparency, account...",[<1-hop>\n\n1 \ndevelopment priorities. These ...,"The principles of transparency, accountability...",multi_hop_abstract_query_synthesizer
8,How does Senator PIA S. CAYETANO's proposed Ph...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,Senator PIA S. CAYETANO's proposed Artificial ...,multi_hop_specific_query_synthesizer
9,How do the penalties imposed by the DOST for u...,"[<1-hop>\n\n1 \nendorsements, voice recordings...",The penalties imposed by the DOST for using AI...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [71]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/20 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/60 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [72]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,How does the European Union's AI Act of 2024 i...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,"The European Union's AI Act of 2024, as the wo...",single_hop_specifc_query_synthesizer
1,What is the significance of Georgetown Univers...,[AI presents enormous opportunities for the Ph...,The context references Georgetown University i...,single_hop_specifc_query_synthesizer
2,What does Section 10 of the Artificial Intelli...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The provided context does not include the spec...,single_hop_specifc_query_synthesizer
3,What are the concerns regarding Artificial Sup...,"[1 \na) Promote innovation, technological adva...",The provided context mentions that the Act sha...,single_hop_specifc_query_synthesizer
4,what prereq for AI license and foundation mode...,[<1-hop>\n\n1 \nSec. 12. Contents of AI Regist...,The context explains that registration with th...,multi_hop_abstract_query_synthesizer
5,How do the development priorities promote inno...,[<1-hop>\n\n1 \ndevelopment priorities. These ...,The development priorities aim to promote inno...,multi_hop_abstract_query_synthesizer
6,How does the proposed Philippine AI regulation...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,The proposed Philippine AI regulation emphasiz...,multi_hop_abstract_query_synthesizer
7,How does the Philippine AI policy ensure respo...,[<1-hop>\n\n1 \nSec. 8. NAICSecretariat. - The...,"The Philippine AI policy, as outlined by the N...",multi_hop_abstract_query_synthesizer
8,How does the Senate's Artificial Intelligence ...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,The Senate's Artificial Intelligence Regulatio...,multi_hop_specific_query_synthesizer
9,How do Sections 8 and 16 of the Philippine AI ...,[<1-hop>\n\n1 \nSec. 8. NAICSecretariat. - The...,Section 8 of the Philippine AI Act outlines th...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [73]:
from langsmith import Client

client = Client()

dataset_name = "Philippines AI Bills"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Philippines AI Bills"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [74]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [77]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [78]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [79]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [81]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG"
)

In [82]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [83]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [84]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [85]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [88]:
rag_chain.invoke({"question" : "How much is the penalty for spreading disinformation?"})

'The penalty for using AI to spread disinformation is a fine ranging from One Million Pesos (Php 1,000,000) to Five Million Pesos (Php 5,000,000), or imprisonment for three (3) years to ten (10) years, or both, at the discretion of the court.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [89]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [90]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

* Answer here
</div>

## LangSmith Evaluation

In [91]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'proper-horn-70' at:
https://smith.langchain.com/o/3a267bc8-b03d-5a9d-ad4a-b45fb776d0de/datasets/138f250b-b926-4aaf-9f60-3b3b35c07dfd/compare?selectedSessions=eebd587e-a398-4248-91b6-d8d6cee45cb5




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,how does the DICT relate to the NAIC and what ...,"Based on the provided context, there is no spe...",None,"The DICT is part of the NAIC, which has jurisd...",0,0,0,10.625573,dfcd1390-1144-4c91-af71-fe9269387879,28914d16-872e-4090-827a-da7c96a809b7
1,How does the Philippine Senate's proposed AI R...,Based on the provided context from the Philipp...,None,The Philippine Senate's proposed AI Regulation...,1,1,0,10.953997,b057a0ac-4d34-42df-af7a-27829005698e,e8878400-471f-42e1-90d8-73c81c16889e
2,How do Sections 8 and 16 of the Philippine AI ...,I don't know.,None,Section 8 of the Philippine AI Act outlines th...,0,0,0,1.644154,1a7969ea-c893-4f7d-ab70-983dd3ac1aa4,e04323d7-a640-44c0-ba60-b30f3b9849eb
3,How does the Senate's Artificial Intelligence ...,The Senate's Artificial Intelligence Regulatio...,None,The Senate's Artificial Intelligence Regulatio...,1,0,0,7.420044,96829e35-b157-4ad4-9503-caf8a67b0bb5,3818d08f-01a7-45e1-a54b-173c5c517ef9
4,How does the Philippine AI policy ensure respo...,I don't know.,None,"The Philippine AI policy, as outlined by the N...",0,0,0,1.461356,63e40948-870f-4399-a9c6-5273c74a8b11,847501db-353e-40fd-9564-c45f9c584b80
5,How does the proposed Philippine AI regulation...,I don't know.,None,The proposed Philippine AI regulation emphasiz...,0,0,0,1.549941,644e33c9-3668-44d1-a5c1-4e3e86bf820b,c26b0f46-f6ee-4dce-ba73-9d73976eee4b
6,How do the development priorities promote inno...,The development priorities promote innovation ...,None,The development priorities aim to promote inno...,0,1,0,4.995311,e3a427fb-735f-48fc-8b12-54194d64464e,d1d45b0c-939b-4488-b06a-afcfda7c8b16
7,what prereq for AI license and foundation mode...,The prerequisite for obtaining any business li...,None,The context explains that registration with th...,1,1,0,3.179763,489b769a-4948-418b-945c-61d3e203cdb0,adbfc971-79e8-494f-ac10-ee40837f8833
8,What are the concerns regarding Artificial Sup...,The concerns regarding Artificial Superintelli...,None,The provided context mentions that the Act sha...,1,1,0,3.287605,6194dc4c-0313-4356-88bc-6e999e071207,9d32eb46-e5e3-4887-a0b1-53c219e095a0
9,What does Section 10 of the Artificial Intelli...,I don't know.,None,The provided context does not include the spec...,1,0,0,1.438448,dbfbdb5b-e21e-4990-89f5-5b8ce589d6d4,d99f4c0e-9b19-4d14-a865-10575c63266e


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [96]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [62]:
rag_documents = docs

In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

</div>

In [64]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

</div>

In [92]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG 2"
)

In [93]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [97]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [99]:
empathy_rag_chain.invoke({"question" : "Why is the Philippines AI Bill important?"})

"Thank you for your thoughtful question. Based on the context provided, the Philippines AI Bill is important because it addresses the profound transformation that Artificial Intelligence is bringing to industries, governance systems, and society as a whole. It aims to establish a national framework that ensures AI is used safely, responsibly, and ethically. This is crucial for supporting the country's digital transformation journey while promoting an inclusive, innovative, and secure digital future. Moreover, the Bill recognizes both the enormous opportunities AI offers—such as improving public services, disaster resilience, agriculture, education, and healthcare—and the risks involved, including incomplete or biased data and potential manipulation. By creating guidelines and safeguards, the Bill seeks to maximize AI’s benefits while mitigating its dangers for the Filipino people. If you have any more concerns or thoughts about this, I'm here to listen and help!"

Finally, we can evaluate the new chain on the same test set!

In [100]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'shiny-cart-58' at:
https://smith.langchain.com/o/3a267bc8-b03d-5a9d-ad4a-b45fb776d0de/datasets/138f250b-b926-4aaf-9f60-3b3b35c07dfd/compare?selectedSessions=d95c8ad2-9fdb-45fc-9003-92aa4e1d2e7c




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,how does the DICT relate to the NAIC and what ...,Thank you for your thoughtful question. Based ...,None,"The DICT is part of the NAIC, which has jurisd...",0,0,1,5.898602,dfcd1390-1144-4c91-af71-fe9269387879,7b0317d1-f432-49c7-9ab8-e11fc7ef91c4
1,How does the Philippine Senate's proposed AI R...,Thank you for your thoughtful question. Based ...,None,The Philippine Senate's proposed AI Regulation...,1,0,1,4.309783,b057a0ac-4d34-42df-af7a-27829005698e,0a4826ef-a32d-402e-9757-bef0a4fd637e
2,How do Sections 8 and 16 of the Philippine AI ...,Thank you for your thoughtful question about h...,None,Section 8 of the Philippine AI Act outlines th...,0,0,1,6.172791,1a7969ea-c893-4f7d-ab70-983dd3ac1aa4,073c1b51-d8cb-4e27-b3f1-c313cbfb8d30
3,How does the Senate's Artificial Intelligence ...,Thank you for your thoughtful question about h...,None,The Senate's Artificial Intelligence Regulatio...,1,0,1,3.174318,96829e35-b157-4ad4-9503-caf8a67b0bb5,191192f4-6a41-423c-bb73-1aa9ba813d64
4,How does the Philippine AI policy ensure respo...,Thank you for your thoughtful question. It's c...,None,"The Philippine AI policy, as outlined by the N...",0,0,1,7.591978,63e40948-870f-4399-a9c6-5273c74a8b11,7d859921-807d-4da5-a49f-fdab84e37dad
5,How does the proposed Philippine AI regulation...,Thank you for your thoughtful question. It’s c...,None,The proposed Philippine AI regulation emphasiz...,0,0,1,7.056722,644e33c9-3668-44d1-a5c1-4e3e86bf820b,7e75125f-cb64-498e-842e-ef631804d93a
6,How do the development priorities promote inno...,Thank you for your thoughtful question. Based ...,None,The development priorities aim to promote inno...,0,0,1,5.517145,e3a427fb-735f-48fc-8b12-54194d64464e,7990c448-c85c-4077-892c-1528095f8887
7,what prereq for AI license and foundation mode...,Thank you for your question. I understand you'...,None,The context explains that registration with th...,0,0,1,5.142149,489b769a-4948-418b-945c-61d3e203cdb0,44a43e65-b8ba-4398-9bdf-5c8557845cf9
8,What are the concerns regarding Artificial Sup...,Thank you for your thoughtful question. Based ...,None,The provided context mentions that the Act sha...,1,0,1,3.722460,6194dc4c-0313-4356-88bc-6e999e071207,214efa13-326c-4ae0-93fc-bacb1e1d02a4
9,What does Section 10 of the Artificial Intelli...,Thank you for your thoughtful question about S...,None,The provided context does not include the spec...,1,1,1,5.592875,dbfbdb5b-e21e-4990-89f5-5b8ce589d6d4,f832b275-f6dd-410f-aec4-d595f629fd89


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #4:

Explain why you believe certain metrics changed in certain ways, and provide a screenshot of the difference between the two chains.

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

</div>